In [21]:
import tensorflow as tf
import os
import numpy as np
import cv2

In [4]:
train_images_path = "/media/koriavinash/New Volume1/Research/MNIST_database/newMNIST/train-images/"
test_images_path = "/media/koriavinash/New Volume1/Research/MNIST_database/newMNIST/test-images/"
train_labels_path = "/media/koriavinash/New Volume1/Research/MNIST_database/newMNIST/labels/trainlabels.csv"
test_labels_path = "/media/koriavinash/New Volume1/Research/MNIST_database/newMNIST/labels/testlabels.csv"

In [23]:
def extract_images(folder_path, total_images):
    return [ cv2.imread(folder_path + str(i) + ".jpg") for i in xrange(total_images)]

def dense_to_one_hot(labels_dense, num_classes=10):
    num_labels = labels_dense.shape[0]
    labels_one_hot = np.zeros((num_labels, num_classes))
    for i in xrange(len(labels_dense)):
        labels_one_hot[i][int(labels_dense[i][1])] = 1
    return labels_one_hot

def extract_labels(file_path, one_hot = False):
    labels = np.genfromtxt(file_path, delimiter=",")
    if one_hot:
        return dense_to_one_hot(labels)
    return labels

# print extract_labels(test_labels_path, one_hot = True)
# print extract_images(test_images_path, 10)

In [24]:
class DataSet(object):
    def __init__(self, images, labels, fake_data=False):
        if fake_data:
            self._num_examples = 10000
        else:
            assert images.shape[0] == labels.shape[0], (
                "images.shape: %s labels.shape: %s" % (images.shape,
                                                       labels.shape))
            self._num_examples = images.shape[0]
            # Convert shape from [num examples, rows, columns, depth]
            # to [num examples, rows*columns] (assuming depth == 1)
            assert images.shape[3] == 1
            images = images.reshape(images.shape[0],
                                    images.shape[1] * images.shape[2])
            # Convert from [0, 255] -> [0.0, 1.0].
            images = images.astype(numpy.float32)
            images = numpy.multiply(images, 1.0 / 255.0)
        self._images = images
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0

    @property
    def images(self):
        return self._images

    @property
    def labels(self):
        return self._labels

    @property
    def num_examples(self):
        return self._num_examples

    @property
    def epochs_completed(self):
        return self._epochs_completed

    def next_batch(self, batch_size, fake_data=False):
        if fake_data:
            fake_image = [1.0 for _ in xrange(784)]
            fake_label = 0
            return [fake_image for _ in xrange(batch_size)], [
                fake_label for _ in xrange(batch_size)]
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Shuffle the data
            perm = numpy.arange(self._num_examples)
            numpy.random.shuffle(perm)
            self._images = self._images[perm]
            self._labels = self._labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        end = self._index_in_epoch
        return self._images[start:end], self._labels[start:end]

In [25]:
def read_data_sets(fake_data=False, one_hot=False, train_image_number = 360000, test_image_number = 60000):
    class DataSets(object):
        pass
    data_sets = DataSets()
    if fake_data:
        data_sets.train = DataSet([], [], fake_data=True)
        data_sets.validation = DataSet([], [], fake_data=True)
        data_sets.test = DataSet([], [], fake_data=True)
        return data_sets
    VALIDATION_SIZE = 5000
    train_images = extract_images(train_images_path, train_image_number)
    train_labels = extract_labels(train_labels_path, one_hot=one_hot)
    test_images = extract_images(test_image_path, test_image_number)
    test_labels = extract_labels(local_file, one_hot=one_hot)
    validation_images = train_images[:VALIDATION_SIZE]
    validation_labels = train_labels[:VALIDATION_SIZE]
    train_images = train_images[VALIDATION_SIZE:]
    train_labels = train_labels[VALIDATION_SIZE:]
    data_sets.train = DataSet(train_images, train_labels)
    data_sets.validation = DataSet(validation_images, validation_labels)
    data_sets.test = DataSet(test_images, test_labels)
    return data_sets